In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from platform import python_version

print(python_version())

3.8.10


In [ ]:
!pip install allennlp==1.5.0 allennlp-models==1.5.0
!pip install networkx
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!apt-get install -y graphviz-dev
!pip install pygraphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.1/517.1 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.1/379.1 KB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.6/593.6 KB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 622.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import re
import spacy
from nltk import tokenize
import networkx as nx
import numpy as np
from collections import defaultdict

class DSU:
  # DSU class for efficient check connectivity
  def __init__(self, colors): # the color here is the number of sentences
    self.father = [i for i in range(colors+1)]

  def get_father(self, u):
    if self.father[u] == u:
      return u
    self.father[u] = self.get_father(self.father[u])
    return self.father[u]

  def unite(self, u, v):
    pu = self.get_father(u)
    pv = self.get_father(v)
    if pu == pv: return False
    self.father[pu] = pv
    return True

class Semantic_Graph_Constructor():
  def __init__(self):
    print("Loading sematic role labeling, coreference resolution and spacy models...")
    self.SLR_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")
    self.coref_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")
    self.inDegree = defaultdict(lambda: 0)
    self.ord = []
    self.nlp = spacy.load("en_core_web_sm")
    self.COLORS = {1: '#1E00F0',
              2: '#22182E',
              3: '#89DCC6',
              4: '#01F21D',
              5: '#1815C5',
              6: '#46D012',
              7: '#B0CCA9',
              8: '#943C5B',
              9: '#070C96',
              10: '#6B7212',
              11: '#974E48',
              12: '#0CC610',
              13: '#E82858',
              14: '#77E7A0',
              15: '#E93B03',
              16: '#4995C9',
              17: '#3201B7',
              18: '#80BE97',
              19: '#01C604',
              20: '#A074A0',
              21: '#035C8D',
              22: '#EB22D0',
              23: '#8D489F',
              24: '#68B7A9',
              25: '#F232D7',
              26: '#A67027',
              27: '#EBB891',
              28: '#C7FAC1',
              29: '#427716',
              30: '#04735A',
              31: '#4F4F26',
              32: '#AA7C3B',
              33: '#D4E151',
              34: '#65DBC6',
              35: '#CA62DD',
              36: '#89DD99',
              37: '#06F81F',
              38: '#29AC30',
              39: '#E06332',
              40: '#201B0C',
              41: '#8324A0',
              42: '#6AB686',
              43: '#240E87',
              44: '#E7B5A5',
              45: '#5F644F',
              46: '#27C198',
              47: '#D13A0A',
              48: '#214F1D',
              49: '#327728',
              50: '#F8DEAD',
              51: '#50BA8B',
              52: '#F37969',
              53: '#524D81',
              54: '#3DEE78',
              55: '#A764A7',
              56: '#6F50DF',
              57: '#9C4FBD',
              58: '#241F54',
              59: '#EB2E98',
              60: '#CF9C86',
              61: '#5F0FB1',
              62: '#DA61F2',
              63: '#54B7A5',
              64: '#F1821E',
              65: '#8D8DD6',
              66: '#BB12D2',
              67: '#48B772',
              68: '#8FF143',
              69: '#851CE7',
              70: '#6D81AA',
              71: '#6DAE07',
              72: '#B0D194',
              73: '#75EFCA',
              74: '#BDA529',
              75: '#84AD5F',
              76: '#8DAF43',
              77: '#13537A',
              78: '#9455C5',
              79: '#C5CF28',
              80: '#FE4BEB',
              81: '#09F7D8',
              82: '#4B64F5',
              83: '#B8C98F',
              84: '#ADE234',
              85: '#98F3B7',
              86: '#72344F',
              87: '#AC7940',
              88: '#021B4A',
              89: '#4C8A7C',
              90: '#CF9487',
              91: '#794BAE',
              92: '#C916AD',
              93: '#E9CF3D',
              94: '#666804',
              95: '#F31582',
              96: '#FD2DC1',
              97: '#C3E59A',
              98: '#B6F9E9',
              99: '#E14740',
              100: '#B3329E'
            }

  # Coreference cluster function
  def find_clusters(self, sent):
    '''
    This function is to find the coreference clusters of the input text.
    '''
    pred = self.coref_predictor.predict(
        document= sent
    )
    clusters = pred['clusters']
    document = pred['document']

    n = 0
    sentence_id = 1
    doc = {}
    sentence_index = {}

    for obj in document:
      sentence_index[n] = sentence_id
      doc.update({n :  obj})
      n = n+1
      if obj == '.':
        sentence_id += 1

    clus_all = []
    cluster = []
    clus_one = {}
    for i in range(0, len(clusters)):
      one_cl = clusters[i]
      for count in range(0, len(one_cl)):
        obj = one_cl[count]
        current = []
        for num in range((obj[0]), (obj[1]+1)):
          for n in doc:
            if num == n: current.append(doc[n])
        if not sentence_index[obj[0]] == sentence_index[obj[1]]: print("Bug in find_clusters")

        cluster.append([" ".join(current), obj[0], obj[1], sentence_index[obj[0]]])
      clus_all.append(cluster)
      cluster = []
    return clus_all, document

  # Semantic cluster function
  def get_semantic_role_labels(self, sent):
    sent = self.coref_predictor.coref_resolved(sent)
    SLR_ans = self.SLR_predictor.predict(sentence = sent)

    p1 = re.compile(r'[[](.+?)[]]', re.S)

    relations = []
    for idx in range(len(SLR_ans['verbs'])):
      a = re.findall(p1, SLR_ans['verbs'][idx]['description'])
      try:
        relations.append([a[0].split(':')[1].strip(),
                          a[1].split(':')[1].strip(),
                          a[2].split(':')[1].strip()])
      except: continue
    return relations

  def get_dependency_and_SRL(self, sent):
    doc = self.nlp(sent)
    named_entities = []
    for ent in doc.ents:
      named_entities.append([ent.text, ent.start_char, ent.end_char])

    triples = {}
    for token in doc:
      triples[(token.text, token.head.text)] = token.dep_
      for child in token.children:
        triples[(token.head.text, child)] = token.head.pos_

    for re in self.get_semantic_role_labels(sent):
      triples[(re[0], re[2])] = re[1]

    return triples, named_entities

  def longestCommonSubstring(self, s, t):
    n = len(s)
    m = len(t)
    dp = [[0 for i in range(m + 1)] for j in range(2)]
    res = 0
    for i in range(1,n + 1):
      for j in range(1,m + 1):
        if(s[i - 1] == t[j - 1]):
          dp[i % 2][j] = dp[(i - 1) % 2][j - 1] + 1
          if(dp[i % 2][j] > res): res = dp[i % 2][j]
        else: dp[i % 2][j] = 0
    return res

  def get_sentence_index(self, rationale, sentences):
    lengths = []
    for idx in range(len(sentences)):
      lengths.append(
          self.longestCommonSubstring(rationale, sentences[idx])
      )
    return np.argmax(np.array(lengths)) + 1

  def get_name_entities_and_coref_words(self, rationale, nodes_in_sentence, sentence_index):
    nodes = []
    for node in nodes_in_sentence[sentence_index]:
      if node in rationale:
        nodes.append(node)

    idx_entities = {}
    for node in nodes:
      idx_entities[node] = rationale.find(node)
    sorted_nodes = [k for k, v in sorted(idx_entities.items(), key=lambda item: item[1])]
    return sorted_nodes

  def get_relevant_nodes(self, rationales, sentences, nodes_in_sentence):
    relevant_nodes = []
    for rationale in rationales:
      idx_sentence = self.get_sentence_index(rationale, sentences)
      sorted_nodes = self.get_name_entities_and_coref_words(rationale,
                                                      nodes_in_sentence,
                                                      idx_sentence)
      sorted_nodes.append(idx_sentence)
      relevant_nodes.append(sorted_nodes)
    return relevant_nodes

  def sentence_index_from_node(self, node_name):
    return int(node_name[-2:].strip())

  # Construct NetworkX graph
  def contruct_networkx_graph(self, isTest, graph_triples, test_ID):
    G = nx.DiGraph()
    for triple in graph_triples:
      head = list(triple)[0]
      tail = list(triple)[1]
      attribute = list(triple)[2]

      head_idx = self.sentence_index_from_node(head)
      tail_idx = self.sentence_index_from_node(tail)
      G.add_node(head, color = self.COLORS[int(head_idx)])
      G.add_node(tail, color = self.COLORS[int(tail_idx)])

      # print("Aattribute: ", str(attribute))
      if 'label' in attribute: G.add_edge(head, tail, label=attribute['label'], color=attribute['color'])
      else: G.add_edge(head, tail, color=attribute['color'])

    pref = "test"
    if not isTest:  pref = "result"

    G_undirected = G.to_undirected()
    return G, G_undirected

  # Construct normal graph for further processing
  # Doing TopoSort to determine with the same sentence, which node go first
  def contruct_normal_graph(self, graph_triples, test_ID):
    G = defaultdict(lambda : [])
    self.inDegree = defaultdict(lambda: 0)

    for triple in graph_triples:
      # print(list(triple), end="\n")
      head = list(triple)[0]
      tail = list(triple)[1]
      attribute = list(triple)[2]

      head_idx = self.sentence_index_from_node(head)
      tail_idx = self.sentence_index_from_node(tail)
      head_color = self.COLORS[int(head_idx)]
      tail_color = self.COLORS[int(tail_idx)]

      u, v = (head, head_color), (tail, tail_color)
      G[u].append(v)
      self.inDegree[u] = self.inDegree[v] = 0
    return G

  def topoSort(self, G):
    self.ord = []
    live_nodes = [key for key in self.inDegree.keys()]

    for key, key_adjs in G.items():
      for key_adj in key_adjs:
        self.inDegree[key_adj] += 1

    def __comparator(u):
      return self.inDegree[u]

    while len(live_nodes) > 0:
      live_nodes.sort(key = __comparator)
      self.ord.append(live_nodes[0])

      for key_adj in G[live_nodes[0]]:
        self.inDegree[key_adj] -= 1
      live_nodes.pop(0)
    return self.ord

  # Get added edges
  def name_to_number(self):
    d = defaultdict(lambda: 0)
    for key, val in self.COLORS.items():
      d[val] = key
    return d

  def get_added_edges(self, G, head_and_tail, tot_sentences):
    def __comparator(p):
      return p[1] - p[0]

    D = self.name_to_number()
    pairs = [[l, r] for l in range(1, tot_sentences+1) for r in range(l+1, tot_sentences+1)]
    pairs.sort(key = __comparator)
    added_edges = []
    dsu = DSU(tot_sentences)

    def go(G, dsu):
      for key, vals in G.items():
        for val in vals:
            dsu.unite(D[key[1]], D[val[1]])

    go(G, dsu)
    # try to prioritize the pair with smaller R - L
    for pair in pairs:
      u, v = pair[0], pair[1]
      cu, cv = self.COLORS[u], self.COLORS[v]

      if head_and_tail[cu] == []: continue
      if head_and_tail[cv] == []: continue

      if dsu.unite(u, v):
        tail_u = head_and_tail[cu][1][0]
        head_v = head_and_tail[cv][0][0]

        added_edges.append((tail_u, head_v, {'label' : 'added_edge', 'color': 'blue'}))
    return added_edges

  # Get head && tail w.r.t difference sentences (colors)
  def get_head_and_tail(self, ord):
    head_and_tail = defaultdict(lambda : [])
    D = self.name_to_number()

    # get head and tail of differences node
    for node in ord:
      if head_and_tail[node[1]] == []: head_and_tail[node[1]] = [node, node]
      else: head_and_tail[node[1]][1] = node
    return head_and_tail

  def just_a_testing_function(self):
    print("Testing SLR and COREF functions...")
    slr_output = self.SLR_predictor.predict(
      sentence="Did Uriah honestly think he could beat the game in under three hours?."
    )
    coref_output = self.coref_predictor.predict(
        document="Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers."
    )
    print("slr_output: ", slr_output)
    print("coref_output: ", coref_output)

    print("Testing semantic role labeling...")
    print(self.get_semantic_role_labels('I go to school, buy the foods, and go back to my room.'))

    print("Testing dependency parsing...")
    print(self.get_dependency_and_SRL("Autonomous cars shift insurance liability toward manufacturers"))

  # Main function to build the graph
  def build_linking_graph(self, selected_context, context_ID = 0):
    self.just_a_testing_function()

    sentences = tokenize.sent_tokenize(selected_context)
    print("Number of tokens in context: ", len(sentences))
    G = nx.DiGraph()

    answers = []
    node_attributes = {}
    nodes_in_sentence = {}

    clusters, document = self.find_clusters(selected_context)
    for cluster in clusters:
      for idx1 in range(len(cluster)-1):
        ele1 = cluster[idx1]
        ele2 = cluster[idx1+1]

        if ele1[3] not in nodes_in_sentence: nodes_in_sentence[ele1[3]] = []
        if ele2[3] not in nodes_in_sentence: nodes_in_sentence[ele2[3]] = []

        G.add_node(str(ele1[0]) + ' ' + str(ele1[3]), color = self.COLORS[int(ele1[3])])
        nodes_in_sentence[ele1[3]].append(str(ele1[0]))
        G.add_node(str(ele2[0]) + ' ' + str(ele2[3]), color = self.COLORS[int(ele2[3])])
        nodes_in_sentence[ele2[3]].append(str(ele2[0]))
        G.add_edge(str(ele1[0]) + ' ' + str(ele1[3]), str(ele2[0]) + ' ' + str(ele2[3]), label="Coreference", color='red')

    for idx in range(len(sentences)):
      sent = sentences[idx]
      try: triples, named_entities = self.get_dependency_and_SRL(sent)
      except: continue

      named_entitites_text = [ent[0] for ent in named_entities]
      if idx+1 not in nodes_in_sentence:
        nodes_in_sentence[idx+1] = []
      nodes_in_sentence[idx+1].extend(named_entitites_text)

      idx_entities = {}
      for ent in nodes_in_sentence[idx+1]:
        try: idx_entities[ent] = sent.find(ent)
        except: continue

      sorted_entities = [k for k, v in sorted(idx_entities.items(), key=lambda item: item[1])]
      for ent in named_entities:
        if not G.has_node(ent[0] + ' ' + str(idx+1)):
          G.add_node(ent[0] + ' ' + str(idx+1), color = self.COLORS[int(idx + 1)])

      for loop_idx in range(len(sorted_entities) - 1):
        G.add_edge(sorted_entities[loop_idx] + ' ' + str(idx+1), sorted_entities[loop_idx+1] + ' ' + str(idx+1), label='Same Sentence', color='green')

    list_edges = list(G.edges(data=True))

    G = self.contruct_normal_graph(list_edges, context_ID)
    ord =  self.topoSort(G)
    head_and_tail = self.get_head_and_tail(ord)
    added_edges = self.get_added_edges(G, head_and_tail, 100)

    print("Added edges: ", str(added_edges))

    new_graph_triples = list_edges + added_edges
    G, G_undirected = self.contruct_networkx_graph(False, new_graph_triples, context_ID)

    GRAPH_FOLDER = "/content/graphs"
    A = nx.nx_agraph.to_agraph(G)
    A.layout(prog='fdp')
    nx.nx_agraph.write_dot(G, 'my_graph.dot')
    A.draw(f"{GRAPH_FOLDER}/{context_ID}.png")

    A_undirected = nx.nx_agraph.to_agraph(G_undirected)
    A_undirected.layout(prog='fdp')
    nx.nx_agraph.write_dot(G_undirected, 'my_graph_undirected.dot')
    A_undirected.draw(f"/content/test_graph.png")

    return new_graph_triples


In [ ]:
semantic_graph_constructor = Semantic_Graph_Constructor()

Loading sematic role labeling, coreference resolution and spacy models...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
downloading: 100%|##########| 405972254/405972254 [00:18<00:00, 21393562.09B/s]


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

/usr/local/lib/python3.8/dist-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
downloading: 100%|##########| 1345986155/1345986155 [01:08<00:00, 19733342.71B/s]


Downloading:   0%|          | 0.00/414 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
selected_context = """
  One day Mary took a walk to the park.
  The park was very close to her house.
  One her way to the park she passed her friend Kim's house.
  Mary stopped by and asked if Kim wanted to play.
  Kim said yes.
  Mary and Kim walked together to the park.
  John's house was three houses down.
  Mary and Kim stopped by to ask John if he wanted to play at the park.
  John said no.
  He was afraid of being chased by a squirrel.
  Mary worried that John didn't like her, but John thought she was a good friend.
  So Mary and Kim went to the park to play.
  They loved the park.
  They loved the flowers, and the swings!
  Soon it was dinnertime and the girls went home.
  What a lovely day at the park.
"""

In [ ]:
semantic_graph_constructor.build_linking_graph(selected_context, context_ID=0)

Testing SLR and COREF functions...
slr_output:  {'verbs': [{'verb': 'Did', 'description': '[V: Did] Uriah honestly think he could beat the game in under three hours ? .', 'tags': ['B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'verb': 'think', 'description': 'Did [ARG0: Uriah] [ARGM-ADV: honestly] [V: think] [ARG1: he could beat the game in under three hours] ? .', 'tags': ['O', 'B-ARG0', 'B-ARGM-ADV', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O', 'O']}, {'verb': 'could', 'description': 'Did Uriah honestly think he [V: could] beat the game in under three hours ? .', 'tags': ['O', 'O', 'O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'verb': 'beat', 'description': 'Did Uriah honestly think [ARG0: he] [ARGM-MOD: could] [V: beat] [ARG1: the game] [ARGM-TMP: in under three hours] ? .', 'tags': ['O', 'O', 'O', 'O', 'B-ARG0', 'B-ARGM-MOD', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-

FileNotFoundError: ignored